In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [5]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 3.835409164428711, Val Loss - 3.713534116744995, Val Accuracy - 0.333
Epoch 20, Train Loss - 0.3688559830188751, Val Loss - 1.7009774446487427, Val Accuracy - 0.695
Epoch 40, Train Loss - 0.14035271108150482, Val Loss - 2.45943546295166, Val Accuracy - 0.703
Epoch 60, Train Loss - 0.11334230750799179, Val Loss - 2.403428554534912, Val Accuracy - 0.731
Epoch 80, Train Loss - 0.12440112233161926, Val Loss - 2.9775681495666504, Val Accuracy - 0.725
Epoch 100, Train Loss - 0.03764927014708519, Val Loss - 2.7373428344726562, Val Accuracy - 0.734
Epoch 120, Train Loss - 0.0729769766330719, Val Loss - 2.7915072441101074, Val Accuracy - 0.703
Epoch 140, Train Loss - 0.04647619649767876, Val Loss - 3.0602614879608154, Val Accuracy - 0.712
Epoch 160, Train Loss - 0.02159230224788189, Val Loss - 3.2196319103240967, Val Accuracy - 0.72
Epoch 180, Train Loss - 0.10485251992940903, Val Loss - 2.886941432952881, Val Accuracy - 0.742
Epoch 200, Train Loss - 0.03181927278637886, V

In [6]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.745
Initial Loss: 3.035961866378784


In [7]:
from collections import defaultdict 
from torch_geometric.utils import to_networkx

# Step 1: Initialize and run Metattack
attacker = Metattack(data, device=device)
attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask,
                         lambda_=0.)
attacker.reset()

#200% perturbation (Large Pool)
attacker.attack(num_budgets=2.0)

# Collect added edges from the single run
edges_to_add = list(attacker._added_edges.keys())


print(f"Number of added edges: {len(edges_to_add)}")
print(f"Sample: {edges_to_add[:5]}")


Peturbing graph...:   0%|          | 0/10556 [00:00<?, ?it/s]

Number of added edges: 724
Sample: [(21, 2410), (68, 2505), (68, 136), (131, 134), (57, 2699)]


In [10]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [11]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.3
budget = int(ptb_rate * initial_edge_count)

In [12]:
print(len(edges_to_add))
print(budget)

724
791


In [16]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [17]:
splits = [0, 0.5, 0.7, 0.9]
split_dic = defaultdict(list)

In [18]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.1500
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/791 [00:00<?, ?it/s]

Final Accuracy: 0.6990
Accuracy Drop: 0.0460
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0750
Phase 1: Added 395 edges out of requested 395.


Peturbing graph...:   0%|          | 0/425 [00:00<?, ?it/s]

Final Accuracy: 0.6980
Accuracy Drop: 0.0470
0.7

--- Running split: 0.7 ---
Second phase perturbation rate: 0.0450
Ran out of candidate edges in Phase 1. Moving to Phase 2.
Phase 1: Added 551 edges out of requested 553.


Peturbing graph...:   0%|          | 0/262 [00:00<?, ?it/s]

Final Accuracy: 0.6970
Accuracy Drop: 0.0480
0.9

--- Running split: 0.9 ---
Second phase perturbation rate: 0.0150
Ran out of candidate edges in Phase 1. Moving to Phase 2.
Phase 1: Added 551 edges out of requested 711.


Peturbing graph...:   0%|          | 0/87 [00:00<?, ?it/s]

Final Accuracy: 0.7140
Accuracy Drop: 0.0310


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
